In [22]:
#hide
#from utils import *
from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai2.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG
import torch
import torch.nn as nn
from torch import optim

from sklearn.model_selection import train_test_split
pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

In [3]:
df = pd.read_csv('/home/dev/course-v3/nbs/dl1/omdena1/model/nyc_data/nyc-rolling-sales-clean-1.csv', sep = ';', low_memory=False)

In [4]:
df

,Land_sqf,Gross_sqf,Year_built,NBH_level,SALE_PRICE,YEARLY_RENT
0,2400,1552,1930,2,220485,10900
1,2742,1207,1925,2,223372,8100
2,5610,1320,1910,5,362981,19000
3,1758,1537,1910,5,245135,9700
4,1317,1339,1920,4,216477,12800
...,...,...,...,...,...,...
286,4000,1562,1945,6,354597,21100
287,2000,1548,1930,6,277394,20200
288,1200,450,1987,6,154983,9500
289,3411,1960,1920,5,311596,23400


In [15]:
X = df[['Land_sqf', 'Gross_sqf', 'Year_built', 'NBH_level']]
Y = df[['SALE_PRICE', 'YEARLY_RENT']]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, shuffle=True)

type(X_train.values)

numpy.ndarray

## Convert dataframe to train/test tensors

In [18]:
#Convert all into tensors

x_train_tensor = torch.Tensor(X_train.values)
y_train_tensor = torch.Tensor(y_train.values)
x_test_tensor = torch.Tensor(X_test.values)
y_test_tensor = torch.Tensor(y_test.values)

# X_train = torch.from_numpy(X_train.to_numpy())
# y_train = torch.squeeze(torch.from_numpy(y_train.to_numpy()).float())

# X_test = torch.from_numpy(X_test.to_numpy()).float()
# y_test = torch.squeeze(torch.from_numpy(y_test.to_numpy()).float())

print(x_train_tensor.shape, y_train_tensor.shape)
print(x_test_tensor.shape, y_test_tensor.shape)

torch.Size([232, 4]) torch.Size([232, 2])
torch.Size([59, 4]) torch.Size([59, 2])


## Prep with Dataset and Dataloader

In [29]:
class Dataset():
    def __init__(self, x, y): self.x, self.y = x, y
    def __len__(self): return len(self.x)
    def __getitem__(self, i): return self.x[i], self.y[i]

In [33]:
train_ds = Dataset(x_train_tensor, y_train_tensor)
valid_ds = Dataset(x_test_tensor, y_test_tensor)

In [34]:
train_dl = DataLoader(train_ds, 32)
valid_dl = DataLoader(valid_ds, 32)

## Model and optimiser (SGD)

In [24]:
def get_model():
    model = nn.Sequential(nn.Linear(4, 3), nn.ReLU(), nn.Linear(3,2))
    return model, optim.SGD(model.parameters(), lr = 0.01)

In [25]:
loss_func = nn.MSELoss()

In [26]:
model, opt = get_model()

## Accuracy

# Train for 1000 epochs

In [42]:
for epoch in range(100):
    model.train()
    for xb, yb in train_dl:
        loss = loss_func(model(xb), yb)
        print(f'In Training. Epoch # {epoch}- Loss = {loss}')
        loss.backward()
        opt.step()
        opt.zero_grad()
        

In Training. Epoch # 0- Loss = 138125535870976.0
In Training. Epoch # 0- Loss = 135089119821824.0
In Training. Epoch # 0- Loss = 132815295348736.0
In Training. Epoch # 0- Loss = 129747329744896.0
In Training. Epoch # 0- Loss = 126684657352704.0
In Training. Epoch # 0- Loss = 124643448979456.0
In Training. Epoch # 0- Loss = 121949581737984.0
In Training. Epoch # 0- Loss = 119356914663424.0
In Training. Epoch # 1- Loss = 117634146238464.0
In Training. Epoch # 1- Loss = 115027730235392.0
In Training. Epoch # 1- Loss = 113121486176256.0
In Training. Epoch # 1- Loss = 110478462287872.0
In Training. Epoch # 1- Loss = 107835849441280.0
In Training. Epoch # 1- Loss = 106132693581824.0
In Training. Epoch # 1- Loss = 103823502737408.0
In Training. Epoch # 1- Loss = 101604497817600.0
In Training. Epoch # 2- Loss = 100184616861696.0
In Training. Epoch # 2- Loss = 97945949044736.0
In Training. Epoch # 2- Loss = 96350427414528.0
In Training. Epoch # 2- Loss = 94071628496896.0
In Training. Epoch # 2-

## Test

In [43]:
model.eval()
pred = model(x_test_tensor)
loss = loss_func(pred, y_test_tensor)
print(loss)

tensor(6.0918e+09, grad_fn=<MseLossBackward>)


###### 